**Hybrid Matching algorithm**


This project demonstrates a hybrid fingerprint matching approach that combines:

1.Minutiae features (Level 2 features: terminations and bifurcations).

2.Texture features (Level 3 features: ridge patterns using Local Binary Patterns).

Matching is performed using a score-level fusion of both feature types to improve accuracy over using minutiae or texture alone.



In [17]:
# Importing libraries
import cv2
import numpy as np
import matplotlib
import sklearn
import skimage
from skimage.feature import local_binary_pattern
from scipy.spatial.distance import euclidean

In [ ]:
# Processing & Minnutiae extraction
 
# Convert images to grayscale.
# Apply Gaussian blur to reduce noise.
# Use binary thresholding to prepare the image for ridge detection.

def extract_minutiae(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    _, binary = cv2.threshold(blur, 127, 255, cv2.THRESH_BINARY_INV)
    
    skeleton = cv2.ximgproc.thinning(binary)

    # Extract endpoints & bifurcations
    minutiae_points = []

    for y in range(1, skeleton.shape[0] - 1):
        for x in range(1, skeleton.shape[1] - 1):
            if skeleton[y, x] == 255:
                window = skeleton[y-1:y+2, x-1:x+2]
                if np.sum(window) == 255 * 2:
                    minutiae_points.append(('termination', (x, y)))
                elif np.sum(window) >= 255 * 4:
                    minutiae_points.append(('bifurcation', (x, y)))

    return minutiae_points


In [ ]:
# Texture Feature Extraction (Using Local Binary Patterns)

def extract_texture(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    radius = 3
    n_points = 8 * radius

    lbp = local_binary_pattern(gray, n_points, radius, method='uniform')
    hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, n_points + 3), density=True)

    return hist


In [ ]:
# Minutiae Matching using simple count comparison

def minutiae_similarity(minutiae1, minutiae2):
    min_count = min(len(minutiae1), len(minutiae2))
    max_count = max(len(minutiae1), len(minutiae2))
    if max_count == 0:
        return 0
    return min_count / max_count


In [ ]:
# Texture Matching using histogram distance

def texture_similarity(hist1, hist2):
    distance = euclidean(hist1, hist2)
    return 1 / (1 + distance)  # Higher is better (normalized)


In [ ]:
# Fusion matching

def hybrid_score(minutiae_score, texture_score, w1=0.6, w2=0.4):
    return w1 * minutiae_score + w2 * texture_score


In [18]:
# Matching Function:

def match_fingerprints(img1, img2):
    # Minutiae
    minutiae1 = extract_minutiae(img1)
    minutiae2 = extract_minutiae(img2)
    min_score = minutiae_similarity(minutiae1, minutiae2)

    # Texture
    tex1 = extract_texture(img1)
    tex2 = extract_texture(img2)
    tex_score = texture_similarity(tex1, tex2)

    # Hybrid Score
    final_score = hybrid_score(min_score, tex_score)

    return final_score


In [19]:
# main execution

img1 = cv2.imread('finger1.tif')
img2 = cv2.imread('finger2.tif')

score = match_fingerprints(img1, img2)
print(f"Matching Score: {score:.4f}")

if score > 0.5:
    print("FINGERPRINT MATCHED")
else:
    print("FINGERPRINT NOT MATCHED")


Matching Score: 0.3908
FINGERPRINT NOT MATCHED
